In [2]:
#settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#show python version
import platform
platform.python_version()



'3.8.5'

In [3]:
import scrapy;
import bs4;
import scrapydo
scrapydo.setup()

In [4]:
import scrapy
from scrapy.crawler import CrawlerProcess

Setup a pipeline

In [5]:
import json
import requests

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('fileoutput.txt', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider): 
        line = json.dumps(dict(item)) + "\n"
        image_url = item['download'][0]
        print(f'Downloading: {image_url}')
        image_req = requests.get(image_url)
        filename = os.path.basename(image_url)
        print(f'saving to: {filename}')
        image = open(f'./backgrounds/{filename}', "wb")
        image.write(image_req.content)
        image.close()
        self.file.write(line)
        return item

In [6]:
import logging

class wallpaperSpider(scrapy.Spider):

    name = 'desktopography-2020'
    start_urls = ['https://desktopography.net/exhibition-2020/']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
       
    }

    def parse(self, response):
        # yield {
        #     'title': response.css('title').extract(),
        #     'url': response.url,
        #     #'entries': response.css('div.portfolio-item').extract(),
        # }
        for entry in response.css('div.portfolio-item a.overlay-background::attr(href)'):
            yield response.follow(url=entry.get(), callback=self.parse_downloads)
    
    def parse_downloads(self, response):
        yield {
            'download': response.xpath("//a[text()=\'3840x2160\']/@href").extract()
        }


In [7]:

#Run the crawler
scrapydo.run_spider(wallpaperSpider, settings={
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/RhayvenJonesTwisted-3bj8m7vlhd5wsblcnioohs.jpg
saving to: RhayvenJonesTwisted-3bj8m7vlhd5wsblcnioohs.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/Michal_Krawczyk_Happines-3bjbqmeuq0peino05zr9j4.jpg
saving to: Michal_Krawczyk_Happines-3bjbqmeuq0peino05zr9j4.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/dt2020_flatlogo-sara-lunatiq-3bjbya7nq8nqp53mmekidc.jpg
saving to: dt2020_flatlogo-sara-lunatiq-3bjbya7nq8nqp53mmekidc.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/hunter_final-2-3bjeo928orodytwh22z5ds.jpg
saving to: hunter_final-2-3bjeo928orodytwh22z5ds.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/Nikola-Angelkoski-Covenant-copy-3bj8mkfxmmgqp68j6568zk.jpg
saving to: Nikola-Angelkoski-Covenant-copy-3bj8mkfxmmgqp68j6568zk.jpg
Downloading: https://desktopography.net/wp-content/uploads/bfi_thumb/Up_the_Sky-3bj8nc32